In [ ]:
from pyspark.sql.functions import rand
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, lit

In [1]:
! ls -la

total 1856000
drwxrwxr-x  4 big big      4096 May 11 21:44 .
drwxr-xr-x 32 big big      4096 May  9 18:30 ..
-rw-rw-r--  1 big big 758702080 May 11 21:27 2019-Oct.zip
drwxrwxr-x  8 big big      4096 May 11 21:08 .git
-rw-rw-r--  1 big big         6 May  8 20:47 .gitignore
drwxrwxr-x  2 big big      4096 May 11 21:44 .ipynb_checkpoints
-rw-rw-r--  1 big big 570895494 May  8 17:13 local_part.csv
-rw-r--r--  1 big big 171002105 May 11 21:13 local_test.csv
-rw-r--r--  1 big big 399893389 May 11 21:13 local_train.csv
-rw-rw-r--  1 big big      3716 May 11 21:14 Preare_data.ipynb
-rw-rw-r--  1 big big        72 May 11 21:43 Recommender.ipynb


In [7]:
df=spark.read.csv('local_train.csv',inferSchema=True) # Read the dataset

In [8]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: long (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: string (nullable = true)



In [4]:
df.show()

+--------------------+----+--------+-------------------+--------------------+---------+------+---------+--------------------+
|                 _c0| _c1|     _c2|                _c3|                 _c4|      _c5|   _c6|      _c7|                 _c8|
+--------------------+----+--------+-------------------+--------------------+---------+------+---------+--------------------+
|2019-10-01 00:00:...|view| 1307067|2053013558920217191|  computers.notebook|   lenovo|251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:...|view| 1003306|2053013555631882655|electronics.smart...|    apple|588.77|555446831|6ec635da-ea15-4a5...|
|2019-10-01 00:00:...|view|27500014|2053013554692358509|                null|  redmond| 37.98|555217733|74d40a28-41f9-432...|
|2019-10-01 00:00:...|view|12712064|2053013553559896355|                null| triangle| 30.89|515454339|828dbd8e-8683-409...|
|2019-10-01 00:00:...|view| 1003141|2053013555631882655|electronics.smart...|    apple|382.97|551377651|ca11a570-47da-

In [9]:
# Users actions
df.groupBy('_c7').count().orderBy('count',ascending=False).show(10,False)

+---------+-----+
|_c7      |count|
+---------+-----+
|512475445|554  |
|512365995|301  |
|526731152|227  |
|513021392|207  |
|546159478|193  |
|512505687|192  |
|546270188|182  |
|516308435|177  |
|514649263|171  |
|551211823|165  |
+---------+-----+
only showing top 10 rows



In [10]:
# brands actions
df.groupBy('_c5').count().orderBy('count',ascending=False).show(10,False)

+-------+------+
|_c5    |count |
+-------+------+
|null   |427856|
|samsung|369262|
|apple  |288448|
|xiaomi |216395|
|huawei |77701 |
|lucente|46132 |
|lg     |39563 |
|bosch  |39530 |
|oppo   |33985 |
|sony   |31908 |
+-------+------+
only showing top 10 rows

